<a href="https://colab.research.google.com/github/Jellyjellyjinjin/Daycon-car-object-detection-/blob/main/yolov5_FREEZE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!unzip /content/drive/MyDrive/open.zip

In [ ]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
x = glob.glob('/content/train/*.png')

In [ ]:
from re import X
import random
# 리스트를 랜덤하게 섞기

random.shuffle(x)

# 리스트 개수
total_images = len(x)

# 리스트 분할 비율 설정
train_ratio = 0.8  # 훈련 데이터 비율
train_count = int(total_images * train_ratio)

# 리스트를 분할하여 저장할 두 개의 리스트 생성
train = x[:train_count]
valid = x[train_count:]

print("훈련 데이터 리스트:", train)
print("테스트 데이터 리스트:", valid)

훈련 데이터 리스트: ['/content/train/syn_04086.png', '/content/train/syn_03242.png', '/content/train/syn_00019.png', '/content/train/syn_05492.png', '/content/train/syn_04092.png', '/content/train/syn_00742.png', '/content/train/syn_02901.png', '/content/train/syn_00885.png', '/content/train/syn_01383.png', '/content/train/syn_04408.png', '/content/train/syn_04363.png', '/content/train/syn_00711.png', '/content/train/syn_05618.png', '/content/train/syn_01332.png', '/content/train/syn_02296.png', '/content/train/syn_00089.png', '/content/train/syn_02349.png', '/content/train/syn_05039.png', '/content/train/syn_06132.png', '/content/train/syn_04204.png', '/content/train/syn_01677.png', '/content/train/syn_04025.png', '/content/train/syn_02780.png', '/content/train/syn_05825.png', '/content/train/syn_04356.png', '/content/train/syn_06369.png', '/content/train/syn_00685.png', '/content/train/syn_04263.png', '/content/train/syn_06391.png', '/content/train/syn_01438.png', '/content/train/syn_03649.p

In [ ]:
# .png를 .txt로 변경하는 함수
def change_extension(filename):
    if filename.endswith(".png"):
        return filename[:-4] + ".txt"  # .png를 제외한 부분에 .txt를 추가
    else:
        return filename  # 다른 확장자인 경우 변경하지 않음

# 파일 이름 변경 및 결과 출력
train_txt = [change_extension(filename) for filename in train]
print(train_txt)
valid_txt = [change_extension(filename) for filename in valid]
print(valid_txt)

['/content/train/syn_04086.txt', '/content/train/syn_03242.txt', '/content/train/syn_00019.txt', '/content/train/syn_05492.txt', '/content/train/syn_04092.txt', '/content/train/syn_00742.txt', '/content/train/syn_02901.txt', '/content/train/syn_00885.txt', '/content/train/syn_01383.txt', '/content/train/syn_04408.txt', '/content/train/syn_04363.txt', '/content/train/syn_00711.txt', '/content/train/syn_05618.txt', '/content/train/syn_01332.txt', '/content/train/syn_02296.txt', '/content/train/syn_00089.txt', '/content/train/syn_02349.txt', '/content/train/syn_05039.txt', '/content/train/syn_06132.txt', '/content/train/syn_04204.txt', '/content/train/syn_01677.txt', '/content/train/syn_04025.txt', '/content/train/syn_02780.txt', '/content/train/syn_05825.txt', '/content/train/syn_04356.txt', '/content/train/syn_06369.txt', '/content/train/syn_00685.txt', '/content/train/syn_04263.txt', '/content/train/syn_06391.txt', '/content/train/syn_01438.txt', '/content/train/syn_03649.txt', '/conte

In [ ]:
def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_height, image_width, _ = source_image.shape

        target_image_path = f"input/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)

        if type == "test":
            continue

        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[5], line[7])), float(min(line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3])), float(max(line[2], line[4]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")

        target_label_txt = f"input/{type}/{os.path.basename(txt_path)}"
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [ ]:
make_yolo_dataset(train, train_txt, "train")
make_yolo_dataset(valid, valid_txt, "valid")
make_yolo_dataset(sorted(glob.glob("/content/test/*")), None, "test")

100%|██████████| 3400/3400 [09:06<00:00,  6.22it/s]


In [ ]:
# For train_txt
# 파일 경로가 포함된 리스트
file_paths = train

# 변경 전과 변경 후의 경로 패턴
old_pattern = "/content/train/"
new_pattern = "/content/input/train/"

# 변경된 경로를 저장할 빈 리스트
train_input = []

# 각 파일 경로를 변경하고 새로운 리스트에 추가
for file_path in file_paths:
    new_file_path = file_path.replace(old_pattern, new_pattern)
    train_input.append(new_file_path)

# 변경된 파일 경로 리스트 출력
print(train_input)


['/content/input/train/syn_04086.png', '/content/input/train/syn_03242.png', '/content/input/train/syn_00019.png', '/content/input/train/syn_05492.png', '/content/input/train/syn_04092.png', '/content/input/train/syn_00742.png', '/content/input/train/syn_02901.png', '/content/input/train/syn_00885.png', '/content/input/train/syn_01383.png', '/content/input/train/syn_04408.png', '/content/input/train/syn_04363.png', '/content/input/train/syn_00711.png', '/content/input/train/syn_05618.png', '/content/input/train/syn_01332.png', '/content/input/train/syn_02296.png', '/content/input/train/syn_00089.png', '/content/input/train/syn_02349.png', '/content/input/train/syn_05039.png', '/content/input/train/syn_06132.png', '/content/input/train/syn_04204.png', '/content/input/train/syn_01677.png', '/content/input/train/syn_04025.png', '/content/input/train/syn_02780.png', '/content/input/train/syn_05825.png', '/content/input/train/syn_04356.png', '/content/input/train/syn_06369.png', '/content/i

In [ ]:
# For valid_txt
# 파일 경로가 포함된 리스트
file_paths = valid

# 변경 전과 변경 후의 경로 패턴
old_pattern = "/content/train/"
new_pattern = "/content/input/valid/"

# 변경된 경로를 저장할 빈 리스트
valid_input = []

# 각 파일 경로를 변경하고 새로운 리스트에 추가
for file_path in file_paths:
    new_file_path = file_path.replace(old_pattern, new_pattern)
    valid_input.append(new_file_path)

# 변경된 파일 경로 리스트 출력
print(valid_input)

['/content/input/valid/syn_06099.png', '/content/input/valid/syn_00606.png', '/content/input/valid/syn_05157.png', '/content/input/valid/syn_00515.png', '/content/input/valid/syn_05119.png', '/content/input/valid/syn_04035.png', '/content/input/valid/syn_03487.png', '/content/input/valid/syn_01379.png', '/content/input/valid/syn_01667.png', '/content/input/valid/syn_04983.png', '/content/input/valid/syn_04867.png', '/content/input/valid/syn_01256.png', '/content/input/valid/syn_06005.png', '/content/input/valid/syn_03512.png', '/content/input/valid/syn_06357.png', '/content/input/valid/syn_03424.png', '/content/input/valid/syn_06053.png', '/content/input/valid/syn_02014.png', '/content/input/valid/syn_01275.png', '/content/input/valid/syn_03673.png', '/content/input/valid/syn_03478.png', '/content/input/valid/syn_03324.png', '/content/input/valid/syn_01466.png', '/content/input/valid/syn_00957.png', '/content/input/valid/syn_04337.png', '/content/input/valid/syn_05996.png', '/content/i

In [ ]:
root_dir  = "/content/input"

# train.txt
with open(os.path.join(root_dir, "train.txt"), 'w') as f:
	f.write('\n'.join(train_input) + '\n')

# valid.txt
with open(os.path.join(root_dir, "valid.txt"), 'w') as f:
	f.write('\n'.join(valid_input) + '\n')



In [ ]:
# 클래스 이름 리스트 생성
with open('/content/classes.txt', 'r') as f:
    lines = f.readlines()
    # Extract class names and store them in a list
    classes = [line.strip().split(',')[1] for line in lines]

In [ ]:
import yaml

yaml_data = {'nc': len(classes),
             'names': classes,
             "path": root_dir,
             "train":os.path.join(root_dir, "train.txt"), # train 경로
             "val":os.path.join(root_dir, "valid.txt") # valid 경로
             }

with open(os.path.join(root_dir, "custom.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15926, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 15926 (delta 20), reused 23 (delta 8), pack-reused 15880
Receiving objects: 100% (15926/15926), 14.66 MiB | 16.41 MiB/s, done.
Resolving deltas: 100% (10915/10915), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.5/609.5 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [ ]:
TRAIN = True
# Number of epochs to train for.
EPOCHS = 70
savepath = "/content/drive/MyDrive/ATL/weight"

In [ ]:
%%time
%cd /content/yolov5

if TRAIN:
    !python train.py --data '/content/input/custom.yaml' --weights yolov5x.pt \
    --img 640 --epochs {EPOCHS} --batch-size 8 --name 'yolov5x_result' \
    --freeze 10 --project=f"{savepath}/freeze" \
    --optimizer Adam --seed 42 --hyp /content/yolov5/data/hyps/hyp.scratch-low.yaml

/content/yolov5
train: weights=yolov5x.pt, cfg=, data=/content/input/custom.yaml, hyp=/content/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=70, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=f/content/drive/MyDrive/ATL/weight/freeze, name=yolov5x_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, seed=42, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-211-g94e943e Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.001, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj

In [ ]:
# 이어서 학습하기 위해 파일 복사
%cp /content/yolov5/f/content/drive/MyDrive/ATL/weight/freeze/yolov5x_result/weights/last.pt /content/drive/MyDrive/ATL/weight/freeze
%cp /content/yolov5/f/content/drive/MyDrive/ATL/weight/freeze/yolov5x_result/weights/best.pt /content/drive/MyDrive/ATL/weight/freeze

In [ ]:
!python detect.py --weights /content/drive/MyDrive/ATL/weight/freeze/best.pt --img 640  --source /content/input/test --save-conf --save-txt

detect: weights=['/content/drive/MyDrive/ATL/weight/freeze/best.pt'], source=/content/input/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-211-g94e943e Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86395471 parameters, 0 gradients, 204.5 GFLOPs
image 1/3400 /content/input/test/064442001.png: 384x640 1 kia_carnival_van_2015_2020, 64.7ms
image 2/3400 /content/input/test/064507368.png: 384x640 1 kia_carnival_van_2015_2020, 1 kia_mohave_suv_2020_, 40.8ms
image 3/3400 /content/input/test/065131036.png: 384x640 1 chevrolet_trailblazer_suv_2021_, 1 kia_mohave_suv_2020_, 1 s

In [ ]:
import glob
import cv2
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
submit = pd.read_csv('/content/sample_submission.csv', header=0)
submit.columns

Index(['file_name', 'class_id', 'confidence', 'point1_x', 'point1_y',
       'point2_x', 'point2_y', 'point3_x', 'point3_y', 'point4_x', 'point4_y'],
      dtype='object')

In [ ]:
infer_txt_list = glob.glob('/content/yolov5/runs/detect/exp/labels/*.txt')

In [ ]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)

    return file_name, int(class_id), confidence, x_min, y_max, x_max, y_max, x_max, y_min, x_min, y_min

In [ ]:

submit = pd.DataFrame(columns=['file_name', 'class_id', 'confidence', 'point1_x', 'point1_y', 'point2_x', 'point2_y', 'point3_x', 'point3_y', 'point4_x', 'point4_y'])

for txt in tqdm(infer_txt_list):
    with open(txt, 'r') as f:
        lines = f.readlines()
        base_file_name = txt.split('/')[-1].split('.')[0]
        img_height, img_width = cv2.imread('/content/yolov5/runs/detect/exp/' + base_file_name + '.png').shape[:2]
        for line in lines:
            file_name, class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y = yolo_to_labelme(line, img_width, img_height, txt)
            submit = submit.append({'file_name':file_name, 'class_id':class_id, 'confidence':confidence, 'point1_x':point1_x, 'point1_y':point1_y, 'point2_x':point2_x, 'point2_y':point2_y, 'point3_x':point3_x, 'point3_y':point3_y, 'point4_x':point4_x, 'point4_y':point4_y}, ignore_index=True)

100%|██████████| 3383/3383 [03:06<00:00, 18.17it/s]


In [ ]:
submit.to_csv('/content/drive/MyDrive/submitfreeze.csv', index=False)